In [1]:
import torch
import pandas as pd

In [14]:
from transformers import BertTokenizer, BertForSequenceClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path_1 = "SJ-14/Intent_Detection_Model"
model_1 = BertForSequenceClassification.from_pretrained(model_path_1,ignore_mismatched_sizes=True)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model_1.to(device)
model_1.eval()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SJ-14/Intent_Detection_Model and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([21]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([21, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [38]:
input_sentence = "What is the finance of this product?"


In [39]:
dataset = pd.read_csv('/content/sofmattress_train.csv')   #replace the file path with your file's path
label_map = {label: idx for idx, label in enumerate(dataset['label'].unique())}

In [40]:
reverse_label_map = {idx: label for label, idx in label_map.items()}



In [44]:
def predict_intent(input_sentence):
  encoding = tokenizer(
    input_sentence,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors="pt"
  ).to(device)

  with torch.no_grad():
    outputs = model_1(
        input_ids=encoding['input_ids'],
        attention_mask=encoding['attention_mask']
    )

    logits = outputs.logits.to(device)

    predicted_class = torch.argmax(logits, dim=-1).item()


    return predicted_class

In [45]:
predicted_class=predict_intent(input_sentence)

In [46]:
predicted_intent = reverse_label_map[predicted_class]

print(f"Input Sentence: {input_sentence}")
print(f"Predicted Intent: {predicted_intent}")

Input Sentence: What is the finance of this product?
Predicted Intent: COD
